# Projet ML: Estimation du coût d'un bien immobilier

## Importation et observation des données

In [221]:
#Importation des bibliothèques utiles et chargement du jeu Sberbank
import pandas as pd
import numpy as np
# read the data
df = pd.read_csv('sberbank.csv')
print (type(df))

<class 'pandas.core.frame.DataFrame'>


Now we are going th have a general view to explore our dataset

In [222]:
print(df.shape)
df

(30471, 292)


,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,30469,2015-06-30,44,27.0,7.0,9.0,1.0,1975.0,2.0,6.0,...,15,5,0,15,26,1,2,84,6,7400000
30467,30470,2015-06-30,86,59.0,3.0,9.0,2.0,1935.0,4.0,10.0,...,313,128,24,98,182,1,82,171,15,25000000
30468,30471,2015-06-30,45,NaN,10.0,20.0,1.0,NaN,1.0,1.0,...,1,1,0,2,12,0,1,11,1,6970959
30469,30472,2015-06-30,64,32.0,5.0,15.0,1.0,2003.0,2.0,11.0,...,22,1,1,6,31,1,4,65,7,13500000


-we have 30471 row and 292 columns in our dataset as well we can see the first and the last 5 rows in our dataset  
-the most interesting column is the last one price_doc which describe the price of the building  
-when viewing the table we can see that the data have many problems which can lead to conficts in our model for example we have a lot of missing values as well we have duplicated column id (one include in our dataset and the other is automatically generated by our code) thus we need to clean the data before excuting our model 

In [223]:
df.describe()

,id,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
count,30471.000000,30471.000000,24088.000000,30304.000000,20899.000000,20899.000000,1.686600e+04,20899.000000,20899.000000,16912.000000,...,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,3.047100e+04
mean,15237.917397,54.214269,34.403271,7.670803,12.558974,1.827121,3.068057e+03,1.909804,6.399301,2.107025,...,32.058318,10.783860,1.771783,15.045552,30.251518,0.442421,8.648814,52.796593,5.987070,7.123035e+06
std,8796.501536,38.031487,52.285733,5.319989,6.756550,1.481154,1.543878e+05,0.851805,28.265979,0.880148,...,73.465611,28.385679,5.418807,29.118668,47.347938,0.609269,20.580741,46.292660,4.889219,4.780111e+06
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000e+00,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+05
25%,7620.500000,38.000000,20.000000,3.000000,9.000000,1.000000,1.967000e+03,1.000000,1.000000,1.000000,...,2.000000,1.000000,0.000000,2.000000,9.000000,0.000000,0.000000,11.000000,1.000000,4.740002e+06
50%,15238.000000,49.000000,30.000000,6.500000,12.000000,1.000000,1.979000e+03,2.000000,6.000000,2.000000,...,8.000000,2.000000,0.000000,7.000000,16.000000,0.000000,2.000000,48.000000,5.000000,6.274411e+06
75%,22855.500000,63.000000,43.000000,11.000000,17.000000,2.000000,2.005000e+03,2.000000,9.000000,3.000000,...,21.000000,5.000000,1.000000,12.000000,28.000000,1.000000,7.000000,76.000000,10.000000,8.300000e+06
max,30473.000000,5326.000000,7478.000000,77.000000,117.000000,6.000000,2.005201e+07,19.000000,2014.000000,33.000000,...,377.000000,147.000000,30.000000,151.000000,250.000000,2.000000,106.000000,218.000000,21.000000,1.111111e+08


here we are exploring the data representation in our dataset via the describe methow which show count,mean,std,min,(25%,50%,75%),max
we can see that we have here 276 columns and not 292 the reason for that is we 16 not numeric column on which we can't execute describe method

In [224]:
df.columns

Index(['id', 'timestamp', 'full_sq', 'life_sq', 'floor', 'max_floor',
       'material', 'build_year', 'num_room', 'kitch_sq',
       ...
       'cafe_count_5000_price_2500', 'cafe_count_5000_price_4000',
       'cafe_count_5000_price_high', 'big_church_count_5000',
       'church_count_5000', 'mosque_count_5000', 'leisure_count_5000',
       'sport_count_5000', 'market_count_5000', 'price_doc'],
      dtype='object', length=292)

## Traitement des données manquante

In this part we are going to get rid of null values, to do this we have two different methods :  
-The first method: we can do this by remplacing the null values with eihter the most frequante value or the median or with the mean value of the column; there's no optimal choise between them we can decide which value to be used based on the situation  
-The second method: we have to drop the column or the row with a lot of misssing value since remplacing them can lead to misleading the model results  

First of all let's see how many data are missing and in which column are they and their type

In [225]:
count=0
for col in df.columns:
    column_missing_values = np.mean(df[col].isnull())
    if column_missing_values>0 :
        print('{} - {} - {}%'.format(col,df[col].dtypes, round(column_missing_values*100)))
        count=count+1
print('{} columns with missing values'.format(count))

life_sq - float64 - 21%
floor - float64 - 1%
max_floor - float64 - 31%
material - float64 - 31%
build_year - float64 - 45%
num_room - float64 - 31%
kitch_sq - float64 - 31%
state - float64 - 44%
preschool_quota - float64 - 22%
school_quota - float64 - 22%
hospital_beds_raion - float64 - 47%
raion_build_count_with_material_info - float64 - 16%
build_count_block - float64 - 16%
build_count_wood - float64 - 16%
build_count_frame - float64 - 16%
build_count_brick - float64 - 16%
build_count_monolith - float64 - 16%
build_count_panel - float64 - 16%
build_count_foam - float64 - 16%
build_count_slag - float64 - 16%
build_count_mix - float64 - 16%
raion_build_count_with_builddate_info - float64 - 16%
build_count_before_1920 - float64 - 16%
build_count_1921-1945 - float64 - 16%
build_count_1946-1970 - float64 - 16%
build_count_1971-1995 - float64 - 16%
build_count_after_1995 - float64 - 16%
metro_min_walk - float64 - 0%
metro_km_walk - float64 - 0%
railroad_station_walk_km - float64 - 0%
railr

As we can see here there's a lot of data missing it variates between 0% approximatally and 47%
but before we solve this problem let's see their distribution in the rows

In [226]:
count=0
for row in df.iloc:
    rows_missing_values = np.mean(row.isnull())
    if rows_missing_values>0 :
        print('{} - {}%'.format(row[0], round(rows_missing_values*100)))
        count=count+1
print('{} row with missing values'.format(count))

1 - 3%
2 - 2%
3 - 2%
4 - 2%
5 - 2%
6 - 2%
7 - 2%
8 - 2%
9 - 2%
10 - 2%
11 - 2%
12 - 4%
13 - 2%
14 - 2%
15 - 2%
16 - 2%
17 - 3%
18 - 2%
19 - 2%
20 - 2%
21 - 6%
22 - 2%
23 - 2%
24 - 2%
25 - 2%
26 - 3%
27 - 2%
28 - 2%
29 - 3%
30 - 2%
31 - 2%
32 - 3%
33 - 2%
34 - 2%
35 - 3%
36 - 3%
37 - 2%
38 - 2%
39 - 2%
40 - 3%
41 - 2%
42 - 3%
43 - 3%
44 - 2%
45 - 2%
46 - 2%
47 - 2%
48 - 3%
49 - 2%
50 - 2%
51 - 2%
52 - 2%
53 - 2%
54 - 2%
55 - 2%
56 - 2%
57 - 3%
58 - 3%
59 - 2%
60 - 2%
61 - 2%
62 - 2%
63 - 2%
64 - 2%
65 - 2%
66 - 3%
67 - 2%
68 - 2%
69 - 2%
70 - 2%
71 - 2%
72 - 4%
73 - 2%
74 - 2%
75 - 2%
76 - 3%
77 - 2%
78 - 3%
79 - 3%
80 - 2%
81 - 2%
82 - 3%
83 - 3%
84 - 2%
85 - 2%
86 - 2%
87 - 2%
88 - 2%
89 - 2%
90 - 2%
91 - 2%
92 - 2%
93 - 3%
94 - 2%
95 - 2%
96 - 2%
97 - 3%
98 - 2%
99 - 3%
100 - 2%
101 - 2%
102 - 2%
103 - 2%
104 - 2%
105 - 2%
106 - 2%
107 - 3%
108 - 2%
109 - 2%
110 - 2%
111 - 2%
112 - 2%
113 - 4%
114 - 2%
115 - 2%
116 - 2%
117 - 3%
118 - 2%
119 - 2%
120 - 2%
121 - 2%
122 - 2%
123 - 2%
1

927 - 2%
928 - 4%
929 - 2%
930 - 2%
931 - 3%
932 - 2%
933 - 3%
934 - 2%
935 - 5%
936 - 2%
937 - 2%
938 - 3%
939 - 3%
940 - 3%
941 - 2%
942 - 2%
943 - 2%
944 - 2%
945 - 3%
946 - 2%
947 - 2%
948 - 2%
949 - 2%
950 - 4%
951 - 3%
952 - 2%
953 - 2%
954 - 2%
955 - 2%
956 - 2%
957 - 2%
958 - 4%
959 - 2%
960 - 2%
961 - 2%
962 - 2%
963 - 3%
964 - 2%
965 - 2%
966 - 2%
967 - 3%
968 - 2%
969 - 2%
970 - 2%
971 - 2%
972 - 2%
973 - 2%
974 - 2%
975 - 2%
976 - 2%
977 - 2%
978 - 2%
979 - 2%
980 - 3%
981 - 2%
982 - 2%
983 - 2%
984 - 2%
985 - 2%
986 - 3%
987 - 3%
988 - 2%
989 - 2%
990 - 2%
991 - 3%
992 - 2%
993 - 2%
994 - 2%
995 - 2%
996 - 2%
997 - 2%
998 - 2%
999 - 2%
1000 - 2%
1001 - 3%
1002 - 3%
1003 - 2%
1004 - 2%
1005 - 2%
1006 - 2%
1007 - 2%
1008 - 2%
1009 - 2%
1010 - 2%
1011 - 2%
1012 - 2%
1013 - 2%
1014 - 3%
1015 - 3%
1016 - 2%
1017 - 2%
1018 - 2%
1019 - 2%
1020 - 2%
1021 - 2%
1022 - 2%
1023 - 2%
1024 - 2%
1025 - 3%
1026 - 3%
1027 - 2%
1028 - 3%
1029 - 2%
1030 - 2%
1031 - 2%
1032 - 2%
1033 - 2%
103

1957 - 2%
1958 - 3%
1959 - 2%
1960 - 2%
1961 - 2%
1962 - 3%
1963 - 3%
1964 - 3%
1965 - 3%
1966 - 2%
1967 - 2%
1968 - 2%
1969 - 2%
1970 - 3%
1971 - 2%
1972 - 2%
1973 - 3%
1974 - 3%
1975 - 2%
1976 - 2%
1977 - 3%
1978 - 2%
1979 - 3%
1980 - 3%
1981 - 2%
1982 - 2%
1983 - 2%
1984 - 2%
1985 - 4%
1986 - 2%
1987 - 2%
1988 - 2%
1989 - 2%
1990 - 2%
1991 - 2%
1992 - 3%
1993 - 2%
1994 - 2%
1995 - 2%
1996 - 2%
1997 - 2%
1998 - 2%
1999 - 3%
2000 - 2%
2001 - 2%
2002 - 3%
2003 - 2%
2004 - 3%
2005 - 2%
2006 - 2%
2007 - 2%
2008 - 3%
2009 - 2%
2010 - 2%
2011 - 3%
2012 - 2%
2013 - 2%
2014 - 3%
2015 - 2%
2016 - 2%
2017 - 2%
2018 - 3%
2019 - 3%
2020 - 2%
2021 - 2%
2022 - 2%
2023 - 2%
2024 - 2%
2025 - 2%
2026 - 2%
2027 - 2%
2028 - 2%
2029 - 3%
2030 - 2%
2031 - 2%
2032 - 2%
2033 - 2%
2034 - 3%
2035 - 2%
2036 - 3%
2037 - 2%
2038 - 3%
2039 - 3%
2040 - 2%
2041 - 3%
2042 - 2%
2043 - 3%
2044 - 2%
2045 - 13%
2046 - 3%
2047 - 2%
2048 - 2%
2049 - 2%
2050 - 2%
2051 - 2%
2052 - 2%
2053 - 2%
2054 - 3%
2055 - 2%
2056 - 2%

2825 - 2%
2826 - 2%
2827 - 2%
2828 - 2%
2829 - 10%
2830 - 2%
2831 - 3%
2832 - 3%
2833 - 5%
2834 - 2%
2835 - 2%
2836 - 3%
2837 - 2%
2838 - 2%
2839 - 2%
2840 - 2%
2841 - 2%
2842 - 2%
2843 - 3%
2844 - 2%
2845 - 2%
2846 - 4%
2847 - 10%
2848 - 3%
2849 - 2%
2850 - 2%
2851 - 2%
2852 - 2%
2853 - 2%
2854 - 2%
2855 - 2%
2856 - 3%
2857 - 2%
2858 - 2%
2859 - 2%
2860 - 2%
2861 - 2%
2862 - 2%
2863 - 2%
2864 - 2%
2865 - 2%
2866 - 3%
2867 - 2%
2868 - 2%
2869 - 2%
2870 - 2%
2871 - 3%
2872 - 5%
2873 - 3%
2874 - 3%
2875 - 2%
2876 - 3%
2877 - 11%
2878 - 2%
2879 - 3%
2880 - 2%
2881 - 2%
2882 - 3%
2883 - 3%
2884 - 2%
2885 - 2%
2886 - 2%
2887 - 2%
2888 - 2%
2889 - 4%
2890 - 2%
2891 - 7%
2892 - 9%
2893 - 2%
2894 - 2%
2895 - 3%
2896 - 2%
2897 - 2%
2898 - 2%
2899 - 3%
2900 - 11%
2901 - 7%
2902 - 3%
2903 - 3%
2904 - 5%
2905 - 2%
2906 - 4%
2907 - 2%
2908 - 3%
2909 - 2%
2910 - 3%
2911 - 2%
2912 - 2%
2913 - 3%
2914 - 3%
2915 - 2%
2916 - 2%
2917 - 2%
2918 - 2%
2919 - 2%
2920 - 2%
2921 - 2%
2922 - 3%
2923 - 2%
2924 -

3727 - 7%
3728 - 11%
3729 - 7%
3730 - 3%
3731 - 5%
3732 - 2%
3733 - 2%
3734 - 6%
3735 - 2%
3736 - 4%
3737 - 2%
3738 - 4%
3739 - 6%
3740 - 7%
3741 - 12%
3742 - 11%
3743 - 9%
3744 - 4%
3745 - 2%
3746 - 3%
3747 - 2%
3748 - 2%
3749 - 2%
3750 - 2%
3751 - 6%
3752 - 7%
3753 - 6%
3754 - 13%
3755 - 4%
3756 - 11%
3757 - 11%
3758 - 2%
3759 - 12%
3760 - 3%
3761 - 2%
3762 - 13%
3763 - 2%
3764 - 2%
3765 - 13%
3766 - 10%
3767 - 2%
3768 - 11%
3769 - 11%
3770 - 10%
3771 - 11%
3772 - 2%
3773 - 3%
3774 - 13%
3775 - 13%
3776 - 14%
3777 - 12%
3778 - 4%
3779 - 4%
3780 - 11%
3781 - 2%
3782 - 2%
3783 - 2%
3784 - 11%
3785 - 2%
3786 - 11%
3787 - 9%
3788 - 2%
3789 - 2%
3790 - 3%
3791 - 13%
3792 - 11%
3793 - 6%
3794 - 2%
3795 - 2%
3796 - 9%
3797 - 13%
3798 - 2%
3799 - 13%
3800 - 2%
3801 - 2%
3802 - 4%
3803 - 4%
3804 - 2%
3805 - 2%
3806 - 13%
3807 - 11%
3808 - 11%
3809 - 9%
3810 - 2%
3811 - 12%
3812 - 3%
3813 - 2%
3814 - 2%
3815 - 11%
3816 - 2%
3817 - 3%
3818 - 2%
3819 - 2%
3820 - 7%
3821 - 7%
3822 - 10%
3823 - 2%

...

25079 - 0%
25080 - 3%
25081 - 1%
25083 - 1%
25084 - 0%
25086 - 0%
25087 - 12%
25090 - 8%
25093 - 2%
25094 - 4%
25095 - 2%
25096 - 13%
25097 - 8%
25099 - 4%
25100 - 0%
25101 - 1%
25102 - 0%
25105 - 14%
25106 - 0%
25109 - 0%
25110 - 10%
25111 - 1%
25112 - 7%
25113 - 1%
25115 - 3%
25116 - 0%
25117 - 0%
25118 - 1%
25120 - 0%
25121 - 3%
25123 - 7%
25124 - 2%
25125 - 0%
25126 - 8%
25127 - 1%
25131 - 0%
25132 - 1%
25134 - 0%
25136 - 1%
25137 - 8%
25140 - 1%
25141 - 0%
25142 - 13%
25143 - 8%
25144 - 1%
25145 - 2%
25146 - 1%
25148 - 2%
25149 - 7%
25150 - 8%
25151 - 1%
25153 - 3%
25154 - 7%
25155 - 1%
25156 - 0%
25157 - 1%
25158 - 2%
25160 - 1%
25161 - 8%
25163 - 3%
25164 - 10%
25165 - 8%
25166 - 8%
25167 - 2%
25168 - 0%
25170 - 0%
25171 - 3%
25172 - 2%
25173 - 0%
25174 - 1%
25178 - 0%
25179 - 0%
25181 - 1%
25182 - 4%
25183 - 0%
25187 - 3%
25190 - 1%
25191 - 0%
25192 - 0%
25194 - 0%
25195 - 0%
25196 - 3%
25198 - 1%
25200 - 12%
25202 - 0%
25204 - 7%
25205 - 0%
25206 - 13%
25208 - 1%
25209 - 3%
25

26330 - 3%
26331 - 7%
26332 - 8%
26334 - 0%
26335 - 1%
26336 - 8%
26337 - 0%
26340 - 7%
26341 - 10%
26342 - 1%
26343 - 1%
26344 - 2%
26345 - 1%
26346 - 7%
26347 - 2%
26348 - 12%
26349 - 7%
26351 - 7%
26352 - 0%
26353 - 0%
26354 - 4%
26355 - 0%
26356 - 0%
26357 - 0%
26358 - 1%
26359 - 0%
26360 - 1%
26361 - 10%
26362 - 0%
26363 - 1%
26365 - 1%
26366 - 1%
26367 - 8%
26368 - 1%
26369 - 0%
26370 - 0%
26371 - 1%
26372 - 7%
26373 - 3%
26374 - 4%
26375 - 8%
26376 - 8%
26379 - 0%
26380 - 0%
26381 - 7%
26382 - 7%
26383 - 8%
26384 - 1%
26385 - 3%
26386 - 8%
26388 - 7%
26389 - 2%
26390 - 0%
26391 - 1%
26392 - 1%
26394 - 0%
26396 - 3%
26397 - 7%
26398 - 8%
26399 - 0%
26401 - 3%
26402 - 3%
26404 - 3%
26405 - 7%
26407 - 0%
26408 - 10%
26411 - 3%
26412 - 8%
26413 - 3%
26414 - 7%
26415 - 1%
26416 - 1%
26417 - 3%
26418 - 2%
26419 - 0%
26420 - 1%
26421 - 3%
26422 - 1%
26423 - 8%
26424 - 3%
26425 - 3%
26426 - 4%
26428 - 9%
26429 - 1%
26431 - 2%
26433 - 12%
26435 - 0%
26436 - 9%
26437 - 10%
26438 - 1%
2643

27463 - 0%
27465 - 1%
27466 - 1%
27468 - 1%
27470 - 3%
27472 - 1%
27473 - 0%
27476 - 2%
27479 - 11%
27480 - 1%
27481 - 0%
27482 - 1%
27483 - 1%
27484 - 0%
27485 - 1%
27486 - 8%
27488 - 8%
27489 - 2%
27491 - 1%
27492 - 0%
27493 - 3%
27495 - 0%
27496 - 0%
27499 - 0%
27501 - 1%
27502 - 7%
27503 - 8%
27504 - 8%
27506 - 0%
27507 - 1%
27508 - 3%
27509 - 2%
27510 - 7%
27511 - 0%
27512 - 0%
27515 - 0%
27516 - 2%
27517 - 7%
27518 - 7%
27520 - 12%
27521 - 1%
27524 - 0%
27526 - 1%
27528 - 1%
27532 - 10%
27533 - 1%
27535 - 3%
27536 - 0%
27537 - 1%
27539 - 3%
27541 - 2%
27542 - 0%
27544 - 1%
27547 - 3%
27548 - 3%
27550 - 0%
27551 - 1%
27554 - 1%
27555 - 8%
27557 - 0%
27558 - 2%
27559 - 0%
27560 - 8%
27561 - 0%
27563 - 4%
27565 - 2%
27568 - 7%
27569 - 0%
27570 - 3%
27571 - 12%
27574 - 0%
27575 - 3%
27576 - 10%
27580 - 1%
27581 - 2%
27582 - 1%
27584 - 1%
27586 - 9%
27587 - 0%
27588 - 1%
27589 - 0%
27594 - 1%
27596 - 0%
27598 - 0%
27599 - 1%
27600 - 1%
27601 - 3%
27602 - 14%
27603 - 8%
27604 - 0%
2760

28693 - 7%
28694 - 7%
28695 - 1%
28696 - 4%
28697 - 1%
28698 - 7%
28699 - 14%
28700 - 8%
28702 - 3%
28703 - 7%
28704 - 7%
28705 - 4%
28706 - 0%
28709 - 10%
28710 - 12%
28711 - 11%
28712 - 11%
28713 - 2%
28714 - 1%
28715 - 11%
28716 - 14%
28717 - 11%
28719 - 8%
28720 - 7%
28721 - 3%
28722 - 3%
28723 - 1%
28724 - 7%
28725 - 1%
28726 - 3%
28728 - 9%
28729 - 7%
28731 - 0%
28732 - 1%
28733 - 1%
28734 - 13%
28735 - 1%
28736 - 0%
28737 - 0%
28738 - 1%
28739 - 8%
28741 - 2%
28743 - 4%
28744 - 8%
28745 - 0%
28747 - 1%
28748 - 0%
28750 - 1%
28751 - 0%
28752 - 13%
28754 - 1%
28755 - 8%
28756 - 1%
28757 - 8%
28758 - 14%
28759 - 2%
28760 - 7%
28761 - 7%
28762 - 0%
28763 - 7%
28764 - 8%
28765 - 0%
28766 - 14%
28767 - 7%
28768 - 1%
28769 - 1%
28770 - 14%
28772 - 10%
28773 - 8%
28775 - 5%
28776 - 0%
28777 - 1%
28779 - 0%
28780 - 1%
28781 - 1%
28783 - 0%
28786 - 0%
28787 - 0%
28788 - 3%
28789 - 1%
28790 - 0%
28791 - 0%
28792 - 4%
28793 - 7%
28794 - 3%
28796 - 0%
28797 - 0%
28798 - 1%
28799 - 1%
28801 -

29707 - 0%
29708 - 7%
29709 - 14%
29710 - 10%
29714 - 1%
29716 - 1%
29717 - 10%
29719 - 2%
29720 - 1%
29723 - 12%
29724 - 7%
29725 - 0%
29726 - 12%
29729 - 12%
29730 - 1%
29731 - 3%
29733 - 7%
29734 - 0%
29735 - 3%
29739 - 2%
29740 - 14%
29741 - 0%
29742 - 7%
29743 - 11%
29745 - 12%
29746 - 2%
29747 - 1%
29748 - 1%
29749 - 1%
29752 - 1%
29753 - 0%
29756 - 7%
29757 - 2%
29758 - 11%
29759 - 12%
29760 - 14%
29761 - 0%
29762 - 12%
29763 - 2%
29764 - 2%
29765 - 3%
29766 - 3%
29767 - 3%
29768 - 0%
29769 - 1%
29771 - 0%
29772 - 1%
29773 - 7%
29774 - 12%
29775 - 1%
29776 - 1%
29777 - 1%
29778 - 3%
29779 - 1%
29780 - 1%
29783 - 0%
29784 - 2%
29786 - 0%
29787 - 8%
29788 - 1%
29790 - 1%
29791 - 1%
29792 - 1%
29793 - 10%
29794 - 1%
29795 - 0%
29796 - 1%
29797 - 2%
29798 - 0%
29800 - 2%
29802 - 2%
29803 - 12%
29804 - 7%
29805 - 8%
29806 - 3%
29807 - 0%
29808 - 4%
29809 - 1%
29814 - 1%
29816 - 2%
29817 - 0%
29818 - 1%
29819 - 0%
29821 - 3%
29822 - 8%
29824 - 0%
29825 - 14%
29826 - 0%
29827 - 8%
2982

As we can see here the list is very long so let's have a better view

In [227]:
count=0
for row in df.iloc:
    rows_missing_values = np.mean(row.isnull())
    if round(rows_missing_values*100)>10 :
        print('{} - {}%'.format(row[0], round(rows_missing_values*100)))
        count=count+1
print('{} rows with missing values'.format(count))

2045 - 13%
2266 - 13%
2734 - 11%
2877 - 11%
2900 - 11%
3007 - 12%
3022 - 12%
3159 - 11%
3170 - 11%
3235 - 11%
3257 - 13%
3260 - 13%
3264 - 13%
3269 - 13%
3277 - 13%
3278 - 13%
3279 - 11%
3283 - 13%
3288 - 13%
3289 - 13%
3290 - 13%
3292 - 13%
3296 - 13%
3298 - 13%
3299 - 12%
3300 - 12%
3301 - 11%
3308 - 13%
3309 - 11%
3313 - 12%
3317 - 13%
3320 - 13%
3323 - 11%
3325 - 11%
3328 - 11%
3329 - 14%
3330 - 11%
3335 - 11%
3339 - 12%
3340 - 13%
3345 - 12%
3349 - 11%
3353 - 13%
3357 - 14%
3362 - 13%
3365 - 13%
3368 - 13%
3372 - 13%
3376 - 13%
3384 - 11%
3385 - 13%
3391 - 14%
3400 - 11%
3401 - 11%
3402 - 12%
3407 - 11%
3415 - 12%
3416 - 14%
3417 - 13%
3423 - 11%
3436 - 11%
3440 - 12%
3441 - 13%
3442 - 12%
3443 - 11%
3445 - 11%
3450 - 11%
3451 - 11%
3454 - 11%
3456 - 13%
3459 - 11%
3461 - 13%
3464 - 11%
3465 - 11%
3478 - 11%
3479 - 13%
3481 - 11%
3491 - 13%
3498 - 11%
3516 - 11%
3525 - 11%
3538 - 11%
3539 - 11%
3544 - 11%
3545 - 11%
3552 - 13%
3555 - 11%
3556 - 11%
3561 - 11%
3563 - 11%
3564 - 11%

6521 - 11%
6527 - 12%
6530 - 11%
6540 - 11%
6547 - 12%
6550 - 11%
6551 - 11%
6565 - 12%
6566 - 13%
6568 - 12%
6573 - 12%
6575 - 11%
6583 - 13%
6588 - 12%
6595 - 14%
6600 - 11%
6601 - 12%
6612 - 12%
6619 - 12%
6620 - 12%
6626 - 12%
6630 - 12%
6632 - 12%
6636 - 12%
6639 - 11%
6643 - 12%
6649 - 12%
6655 - 14%
6658 - 11%
6663 - 12%
6667 - 12%
6671 - 11%
6676 - 11%
6680 - 12%
6697 - 11%
6704 - 12%
6714 - 12%
6720 - 11%
6728 - 11%
6738 - 12%
6745 - 11%
6746 - 13%
6749 - 12%
6756 - 14%
6760 - 12%
6771 - 12%
6775 - 14%
6790 - 12%
6793 - 12%
6814 - 12%
6822 - 12%
6823 - 11%
6839 - 12%
6849 - 11%
6852 - 14%
6853 - 12%
6866 - 12%
6873 - 11%
6875 - 11%
6878 - 11%
6884 - 12%
6885 - 12%
6886 - 12%
6888 - 12%
6890 - 12%
6893 - 12%
6894 - 12%
6909 - 12%
6920 - 11%
6923 - 12%
6927 - 14%
6931 - 11%
6941 - 11%
6943 - 12%
6944 - 12%
6945 - 12%
6956 - 12%
6958 - 11%
6966 - 12%
6967 - 12%
6968 - 11%
6969 - 12%
6974 - 12%
6975 - 12%
6980 - 12%
6983 - 12%
6984 - 12%
6987 - 12%
6988 - 12%
6994 - 12%
6999 - 11%

16742 - 12%
16745 - 13%
16751 - 13%
16782 - 12%
16786 - 12%
16834 - 12%
16836 - 13%
16878 - 12%
16882 - 12%
16916 - 12%
16944 - 11%
16953 - 12%
16972 - 13%
17052 - 11%
17053 - 12%
17093 - 13%
17178 - 12%
17189 - 12%
17252 - 13%
17317 - 12%
17365 - 12%
17472 - 12%
17474 - 13%
17505 - 12%
17513 - 13%
17534 - 13%
17569 - 12%
17570 - 12%
17629 - 12%
17638 - 12%
17673 - 12%
17709 - 13%
17711 - 13%
17753 - 12%
17769 - 13%
17789 - 12%
17878 - 13%
17886 - 12%
17898 - 11%
17913 - 12%
17917 - 12%
17921 - 13%
18006 - 12%
18064 - 13%
18163 - 12%
18198 - 13%
18237 - 12%
18252 - 12%
18276 - 13%
18277 - 14%
18283 - 12%
18337 - 13%
18367 - 11%
18371 - 12%
18454 - 12%
18529 - 12%
18543 - 11%
18567 - 12%
18594 - 13%
18614 - 13%
18622 - 12%
18689 - 13%
18717 - 13%
18753 - 11%
18756 - 11%
18783 - 12%
18814 - 12%
18837 - 12%
18872 - 12%
18911 - 12%
18920 - 12%
18963 - 13%
18983 - 12%
18999 - 13%
19003 - 14%
19025 - 13%
19282 - 12%
19314 - 12%
19315 - 12%
19330 - 12%
19333 - 12%
19350 - 12%
19355 - 13%
1939

Now we have only 2206 rows  
So now the question which data should we get rid of?  
I think it's better to get rid of the rows with huge missing values first because row has less impact on data than the column then we reevalute our data

In [ ]:
#print(df.index[df['id']==2045].tolist())


for row in df.iloc:
    rows_missing_values = np.mean(row.isnull())
    if round(rows_missing_values*100)>10 :
        df=df.drop(df.index[df['id']==row[0]].tolist())

print(df.shape)


now let's see the column missing values rate

In [ ]:
count=0
for col in df.columns:
    column_missing_values = np.mean(df[col].isnull())
    if column_missing_values>0 :
        print('{} - {} - {}%'.format(col,df[col].dtypes, round(column_missing_values*100)))
        count=count+1
print('{} columns with missing values'.format(count))

Now as we can see the missing values rate per column has been sgnificatlly decresed but we there still work to be done  
So now will get rid of the columns with huge rate

In [ ]:
print(df.shape)
count=0
for col in df.columns:
    count=count+1
    column_missing_values = np.mean(df[col].isnull())
    if round(column_missing_values*100)>25 :
        del df[col]
        #df=df.drop(columns=count,axis=1)
        count=count-1
print(df.shape)

In this notebook, we are going to replace the not numeric values by the most frequent value and we are going to use the mediane for the numeric values

In [ ]:
#remplcer les valeurs null dans les colones non nummériques par la valeur la plus fréquente

df_categoric = df.select_dtypes(exclude='number')
categoric_cols = df_categoric.columns.values

for col in categoric_cols:
    missing = df[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  # only do the imputation for the columns that have missing values.
        print(col)
        top = df[col].describe()['top'] # impute with the most frequent value.
        
        df[col] = df[col].fillna(top)
        


#remplcer les valeurs null dans les colones nummériques par la valeur la mediane
df_numeric = df.select_dtypes(include=[np.number])

numeric_cols = df_numeric.columns.values

for col in numeric_cols:
    missing = df[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  # only do the imputation for the columns that have missing values.
        med = df[col].median()
        df[col] = df[col].fillna(med)

        df.head()
print(df.shape)

## Traitement des données aberrantes

Now does all the data that we have are correctly placed?  
Unfortantlly no,this can be caused for a lot of reasons for example the user has entred the data wrongly or the data has been worongly mesured.  
So how we are going to identify the wrong data? well in the most cases that data have either a very low value or very high. so let's have a look about our data. 

In [ ]:
for col in df.columns:
    if len(df[col].unique().tolist())<5:
        print(col)
        print(df[col].unique().tolist())

In [ ]:
Q1=df.price_doc.quantile(0.01)
Q3=df.price_doc.quantile(0.99)
ind=df[(df['price_doc'] < Q1 ) | (df['price_doc'] > Q3 )].index
df.drop(ind,axis=0)
print(df.shape)
len(ind)

df_numeric = df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values

for col in numeric_cols:
    Q1=df.full_sq.quantile(0.25)
    Q3=df.full_sq.quantile(0.75)
    Q=df.full_sq.describe()
    IQR=Q3-Q1
    ind=df[(df[col] < Q1-1.5*IQR ) ].index
    ind2=df[(df[col] > Q3+1.5*IQR ) ].index

    df.loc[ind,col]=Q1
    df.loc[ind2,col]=Q3
print(df.shape)

## Traitement du problème d'incohérence des données 

In this part, we are going to normalize our data for example some strings can have a white spacein the biginnig or in the end thus that make her a whole different string for our model

In [ ]:
df_categoric = df.select_dtypes(exclude='number')
categoric_cols = df_categoric.columns.values

for col in categoric_cols:
    print(col)
    unique=df[col].unique().tolist()
    unique.sort()
    print(unique)

In [ ]:
df_categoric = df.select_dtypes(exclude='number')
categoric_cols = df_categoric.columns.values

for col in categoric_cols:
    #supression des espaces blancs au debut
    df[col] = df[col].str.lstrip()

    #supression des espaces blancs à la fin
    df[col] = df[col].str.rstrip()
    
    #supression des espaces entre les mots
    df[col] = df[col].str.replace(' ','')

    #supression des symboles monétaires
    money_chars = ["$","€","¢","£","₿"]
    df[col]= df[col].replace('money_chars','')

#formater les dates
df['timestamp'] = pd.to_datetime(df['timestamp'],format='%Y-%m-%d')
df.head()
print(df.shape)

## traitement de redondonce des données

In our dataset, all rows must be unique in order to have an accurate resultat so we have to remouve the duplicated rows

In [ ]:
#code ici
#df.drop('price_doc', axis=1).drop_duplicates()
df.drop_duplicates()

print(df.shape)
df.head()


### Transformation des données 

In [ ]:
#code ici
#DFCat=df[categoric_cols]
#DFNum=df[numeric_cols]
#DFCat.head()
#print(DFCat.shape)
#DFCat.head()
#DFCat['culture_objects_top_25']=DFCat['culture_objects_top_25'].replace({"no":0,"yes":1})
#DFCat.replace({"no":0,"yes":1},inplace=True)
from sklearn import preprocessing
#DFCat=df[categoric_cols]
label_encoder = preprocessing.LabelEncoder()
#DFCat['culture_objects_top_25']= label_encoder.fit_transform(DFCat['culture_objects_top_25'])
df=df.replace({"no":0,"yes":1})


df_categoric = df.select_dtypes(exclude='number')
categoric_cols = df_categoric.columns.values

for col in categoric_cols:
    df[col]=label_encoder.fit_transform(df[col])
    

#df = df.drop(columns='product_type')
df.head()

## entrainement des données

In [ ]:
#code ici
from sklearn.model_selection import train_test_split
Data2=df[numeric_cols]
df_y=Data2[['price_doc']]
df_x=Data2.drop(columns=['price_doc'])
print(Data2)

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.2, random_state = 4 )

scaler = preprocessing.StandardScaler()
scaler.fit(x_train) 
xtrainnorm=scaler.transform(x_train)           
xtestnorm=scaler.transform(x_test) 
print(df.shape)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

print(xtrainnorm.mean())
print(xtrainnorm.std())

## model excution

Finnally, we are going to excute our model and test his performance.  
In fact,there's a plenty of model types so we are going to test some of them. 

### Linear Regresion


In [ ]:
from sklearn import linear_model

model = linear_model.LinearRegression()

model.fit(x_train,y_train)

results = model.predict(x_test)
print(results)
print(y_test)
#print(y_test)
r2_score = model.score(x_train,y_train)
r3_score = model.score(x_test,y_test)

print(r2_score*100,'%')
print(r3_score*100,'%')

In [ ]:
df.boxplot(column='full_sq')
Q1=df.full_sq.quantile(0.25)
Q3=df.full_sq.quantile(0.75)
Q=df.full_sq.describe()
IQR=Q3-Q1
print(IQR)
print(Q1)
print(Q3)
ind=df[(df['full_sq'] < Q1-1.5*IQR ) ].index
ind2=df[(df['full_sq'] > Q3+1.5*IQR ) ].index

print(df.shape)
print(len(ind2))
print(len(ind))
df.loc[ind,'full_sq']=Q1
df.loc[ind2,'full_sq']=Q3


In [ ]:
from sklearn import preprocessing
Data=df[numeric_cols].drop(columns=['id','price_doc' ],axis=1)

print(df[numeric_cols].shape)
print(Data.shape)

min_max_scaler = preprocessing.MinMaxScaler()
Dataminmax = min_max_scaler.fit_transform(Data) 
mx=df[numeric_cols].max()
mn=df[numeric_cols].min()
print(mx)
print(mn)
print(Dataminmax.max())
print(Dataminmax.min())

In [ ]:
from sklearn.model_selection import train_test_split
Data2=df[numeric_cols]
df_y=Data2[['price_doc']]
df_x=Data2.drop(columns=['price_doc'])

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.2, random_state = 4 )

print(x_train.shape)
print(x_test.shape)

scaler = preprocessing.StandardScaler()
scaler.fit(x_train) 
xtrainnorm=scaler.transform(x_train)           
xtestnorm=scaler.transform(x_test)           

print(xtrainnorm.mean())
print(xtrainnorm.std())



In [ ]:
from sklearn import linear_model

model = linear_model.LinearRegression()

model.fit(x_train,y_train)

results = model.predict(x_test)
print(results)
print(y_test)
#print(y_test)
r2_score = model.score(x_train,y_train)
r3_score = model.score(x_test,y_test)

print(r2_score*100,'%')
print(r3_score*100,'%')

In [ ]:
#Utilisation de la règle interquartile pour trouver des valeurs aberrantes
df.boxplot(column='female_f')
Q1=df.female_f.quantile(0.25)
Q3=df.female_f.quantile(0.75)
Q=df.female_f.describe()
IQR=Q3-Q1
print(IQR)
print(Q1)
print(Q3)
ind=df[(df['female_f'] < Q1-1.5*IQR ) ].index
ind2=df[(df['female_f'] > Q3+1.5*IQR ) ].index

print(df.shape)
print(len(ind2))
print(len(ind))
df.loc[ind,'female_f']=Q1
df.loc[ind2,'female_f']=Q3

In [ ]:

from sklearn.model_selection import train_test_split
Data2=df[numeric_cols]
df_y=Data2[['price_doc']]
df_x=Data2.drop(columns=['price_doc'])

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.3, random_state = 4 )

print(x_train.shape)
print(x_test.shape)

scaler = preprocessing.StandardScaler()
scaler.fit(x_train) 
xtrainnorm=scaler.transform(x_train)           
xtestnorm=scaler.transform(x_test)           

print(xtrainnorm.mean())
print(xtrainnorm.std())

In [ ]:
from sklearn import linear_model

model = linear_model.LinearRegression()

model.fit(x_train,y_train)

results = model.predict(x_test)
print(results)
print(y_test)
#print(y_test)
r2_score = model.score(x_train,y_train)
r3_score = model.score(x_test,y_test)

print(r2_score*100,'%')
print(r3_score*100,'%')

In [ ]:

df.boxplot(column='office_sqm_500')
Q1=df.office_sqm_500.quantile(0.25)
Q3=df.office_sqm_500.quantile(0.75)
Q=df.office_sqm_500.describe()
IQR=Q3-Q1
print(IQR)
print(Q1)
print(Q3)
ind=df[(df['office_sqm_500'] < Q1-1.5*IQR ) ].index
ind2=df[(df['office_sqm_500'] > Q3+1.5*IQR ) ].index

print(df.shape)
print(len(ind2))
print(len(ind))
df.loc[ind,'office_sqm_500']=Q1
df.loc[ind2,'office_sqm_500']=Q3

In [ ]:
from sklearn.model_selection import train_test_split
Data2=df[numeric_cols]
df_y=Data2[['price_doc']]
df_x=Data2.drop(columns=['price_doc'])

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.3, random_state = 8 )

print(x_train.shape)
print(x_test.shape)

scaler = preprocessing.StandardScaler()
scaler.fit(x_train) 
xtrainnorm=scaler.transform(x_train)           
xtestnorm=scaler.transform(x_test)           

print(xtrainnorm.mean())
print(xtrainnorm.std())



In [ ]:
from sklearn import linear_model

model = linear_model.LinearRegression()

model.fit(x_train,y_train)

results = model.predict(x_test)
print(results)
print(y_test)
#print(y_test)
r2_score = model.score(x_train,y_train)
r3_score = model.score(x_test,y_test)

print(r2_score*100,'%')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Data2=df[numeric_cols]
df_y=Data2[['price_doc']]
df_x=Data2.drop(columns=['price_doc'])


x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.2, random_state = 4 )


sc = StandardScaler()
X_train_std = sc.fit_transform(x_train)
X_test_std = sc.transform(x_test)


### KNN models

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.neighbors import KNeighborsClassifier
kbins = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
ytrain_trans = kbins.fit_transform(y_train)
ytest_trans = kbins.transform(y_test)

model = KNeighborsClassifier(n_neighbors=3)

model.fit(x_train, ytrain_trans)

print(model.score(x_train, ytrain_trans)*100,'%')
print(model.score(x_test, ytest_trans)*100,'%')

### decision tree classifier

In [ ]:
from sklearn.model_selection import RepeatedKFold, RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from numpy import mean
from numpy import std

clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100,max_depth=5, min_samples_leaf=20)
  
    # Performing training
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3,random_state = 10)
n_scores = cross_val_score(clf_gini, x_train, ytrain_trans, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report pipeline performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))